In [35]:
import pandas as pd

In [36]:
movies = pd.read_csv('../data/reduced/movies_m10_rich_pre.csv', index_col='movieId')
ratings = pd.read_csv('../data/reduced/ratings_m10.csv', index_col='userId')

In [37]:
ratings.sample()

,movieId,rating,timestamp
userId,,,
274,1080,3.5,1175629007


In [38]:
movies.sample()

,tmdbId,imdbId,cast,director,keywords,overview,title,genres,year
movieId,,,,,,,,,
92535,80379,2112999,['Louis C.K.'],Louis C.K.,['stand-up'],"Recorded November 10th, 2011 as part of the Ne...",Louis C.K.: Live at the Beacon Theater,['Comedy'],2011


In [39]:
ratings.groupby('userId').value_counts()

userId  movieId  rating  timestamp 
1       1        4.0     964982703     1
        3        4.0     964981247     1
        3793     5.0     964981855     1
        5060     5.0     964984002     1
        3809     4.0     964981220     1
                                      ..
610     1210     5.0     1479544574    1
        1209     5.0     1479544789    1
        1208     5.0     1479542936    1
        1201     5.0     1479542951    1
        168252   5.0     1493846352    1
Name: count, Length: 81116, dtype: int64

In [40]:
#ratings = ratings.head(200)

In [43]:
ratings.index.unique()

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
      dtype='int64', name='userId', length=610)

In [82]:
def train_test_column_split(df, split_column, y_label, train_size):
    df = df.sort_values(by=split_column, ascending=True)   
    train = pd.DataFrame(columns=df.columns)
    test = pd.DataFrame(columns=df.columns)

    for idx in df.index.unique():
        user = ratings.loc[df.index == idx]

        q_user = user[user[split_column].le(user[split_column].quantile(train_size))]
        p_user = user[user[split_column].ge(user[split_column].quantile(train_size))]

        train = pd.concat([train, q_user])
        test = pd.concat([test, p_user])
    train = train.sort_index(ascending=True)
    test = test.sort_index(ascending=True)

    X_labels = [c for c in df.columns if c != y_label]

    X_train = train[X_labels]
    X_test = test[X_labels]
    y_train = train[y_label]
    y_test = test[y_label]

    return (X_train, X_test, y_train, y_test)

In [83]:
ratings = ratings.sort_values(by='timestamp', ascending=True)

In [88]:
X_train, X_test, y_train, y_test = train_test_column_split(ratings, 'timestamp', 'rating', 0.9)

In [89]:
X_train

,movieId,timestamp
1,1278,964983414
1,349,964982563
1,2470,964982588
1,362,964982588
1,2617,964982588
...,...,...
610,2985,1479545763
610,2916,1479545761
610,79695,1479545724
610,592,1479545848


In [90]:
y_train

1      5.0
1      4.0
1      5.0
1      5.0
1      2.0
      ... 
610    5.0
610    5.0
610    3.0
610    4.5
610    5.0
Name: rating, Length: 72991, dtype: float64

In [91]:
X_test

,movieId,timestamp
1,5060,964984002
1,1270,964983705
1,1240,964983723
1,3702,964983737
1,1206,964983737
...,...,...
610,2143,1493879455
610,2968,1493879465
610,168250,1494273047
610,3033,1493879404


In [92]:
y_test

1      5.0
1      5.0
1      5.0
1      5.0
1      5.0
      ... 
610    3.5
610    5.0
610    5.0
610    3.5
610    3.0
Name: rating, Length: 8621, dtype: float64